In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import talib as ta
from datetime import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()
from sklearn.metrics import accuracy_score, r2_score
from datetime import datetime as dt, timedelta

In [2]:
def get_prices(ticker, start_date, end_date):
    start = datetime.strptime(start_date, '%d/%m/%Y')
    end = datetime.strptime(end_date, '%d/%m/%Y')
    f = web.DataReader(ticker, 'morningstar', start, end)
    df = pd.DataFrame(f)
    
    return df

## SQL Connection

In [3]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

def create_connection():
    # Login details: 
    DSN_Name = 'traderDSN' ; Login_ID = 'basic' ; pwd = 'pwd1'

    # The engine does the same job as a connection and a cursor
    engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 

    return engine 

# Build query here:
query = ''' 
SELECT * FROM {schema}.{table_name}

'''.format(schema='stocks', table_name='Adjusted_Fortune_500')
print('Query:', query)
print('='*60)


engine = create_connection()

# Importing a Pandas Dataframe: 
df1 = pd.read_sql(query,engine)

df1.head()

Query:  
SELECT * FROM stocks.Adjusted_Fortune_500




,index,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,date,Name,Ticker
0,0,84.1400,84.5400,83.8200,84.5100,84.5100,10713391,0.0000,1.0000,2018-05-21,Walmart,WMT
1,0,84.2400,84.4400,83.4900,83.6400,83.6400,12043137,0.0000,1.0000,2018-05-18,Walmart,WMT
2,0,87.0400,87.5900,83.8500,84.4900,84.4900,29720738,0.0000,1.0000,2018-05-17,Walmart,WMT
3,0,84.8700,86.6250,84.7400,86.1300,86.1300,13733627,0.0000,1.0000,2018-05-16,Walmart,WMT
4,0,84.0600,84.8800,83.8000,84.5200,84.5200,9322923,0.0000,1.0000,2018-05-15,Walmart,WMT


In [4]:
prices = df1.copy()

prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
prices.head()

,open,high,low,close,volume,date,name,symbol
0,84.1400,84.5400,83.8200,84.5100,10713391,2018-05-21,Walmart,WMT
1,84.2400,84.4400,83.4900,83.6400,12043137,2018-05-18,Walmart,WMT
2,87.0400,87.5900,83.8500,84.4900,29720738,2018-05-17,Walmart,WMT
3,84.8700,86.6250,84.7400,86.1300,13733627,2018-05-16,Walmart,WMT
4,84.0600,84.8800,83.8000,84.5200,9322923,2018-05-15,Walmart,WMT


In [5]:
df1.shape

(2031360, 12)

In [6]:
    %%time 
    
    # Create engine and query 
    engine = create_connection()
    query = ''' 
    SELECT * FROM {schema}.{table_name}
    '''.format(schema='stocks', table_name='Adjusted_Fortune_500')

    # Importing a Pandas Dataframe
    prices = pd.read_sql(query,engine)
    prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
    prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
    
    # Reformat column types
    prices['open']  = prices['open'].astype(float)
    prices['high']  = prices['high'].astype(float)
    prices['low']   = prices['low'].astype(float)
    prices['close'] = prices['close'].astype(float)
    prices['volume']= prices['volume'].astype(float)

    prices['date'] = pd.to_datetime(prices['date'])

    prices.set_index(['symbol', 'name', 'date'], inplace=True)
    prices = prices.sort_index(level=1, ascending=True)

Wall time: 29.4 s


In [7]:
prices.head()

open    high    low    close     volume
symbol name date                                                 
MMM    3M   2000-01-03   96.06   96.44  94.06  30.2529  1086700.0
            2000-01-04   92.88   94.81  90.63  29.0509  1356900.0
            2000-01-05   91.13   95.31  91.13  30.5318  1849700.0
            2000-01-06   94.31  102.50  94.31  32.3108  2987900.0
            2000-01-07  101.10  103.80  99.94  32.9519  2050600.0

In [8]:
    # Calculated columns
    prices  = prices.copy().loc[(prices.loc[:,'open':'volume'] != 0 ).all(axis=1)] # Don't touch this (:
    
    # Group over the symbols. 
    prices_g = prices.groupby(level=['symbol'])
    
    EMA_lengths = [20, 50, 200]  # Calculate many moving averages, of closing prices
    for l in EMA_lengths: 
        prices['EMA_'+str(l)] = prices_g.close.apply(lambda x: ta.EMA(x,l))   

    prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
    prices['RSI'] = prices_g.close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

    def CCI_func(x):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.CCI(high=xh, low=xl, close=xc, timeperiod=5)
   
    prices['CCI'] = prices_g.apply(CCI_func).reset_index(level=1,).drop(columns='symbol')    # created an extra column. 

    prices['Momentum'] = prices_g.close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum changed to prices_g.close

    def Stoch_fast(x, index):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.STOCHF(high=xh, low=xl, close=xc, fastk_period=14, fastd_period=3)[index] # Params 14, 3    
    
    #prices['Stoch_fastk'] = prices_g.apply(Stoch_fast, 0).reset_index(level=1,).drop(columns='symbol')   # Stochastic fast indicator
    #prices['Stoch_fastd'] = prices_g.apply(Stoch_fast, 1).reset_index(level=1,).drop(columns='symbol')

   # def OBV_func(x):
    #    xc = x.close
   #     xv = x.volume
    #    return ta.OBV(xc, xv) 
    #prices['OBV'] = prices_g.apply(OBV_func).reset_index(level=1,).drop(columns='symbol')  # On Balance Volume
    

    def bband_func(x, index):
        xc = x.close
        return ta.BBANDS(xc)[index]
    
    prices['bbandupper'] = prices_g.apply(bband_func, 0).reset_index(level=1,).drop(columns='symbol')
    prices['bbandmiddle'] = prices_g.apply(bband_func, 1).reset_index(level=1,).drop(columns='symbol') 
    prices['bbandlower'] =  prices_g.apply(bband_func, 2).reset_index(level=1,).drop(columns='symbol')


In [9]:
prices['bbandlowerii'] =  prices['bbandlower']/prices['close']
prices['bbandupperii'] =  prices['bbandupper']/prices['close']

In [10]:
prices.head(205)

open    high     low    close     volume     EMA_20  \
symbol name date                                                                
MMM    3M   2000-01-03   96.06   96.44   94.06  30.2529  1086700.0        NaN   
            2000-01-04   92.88   94.81   90.63  29.0509  1356900.0        NaN   
            2000-01-05   91.13   95.31   91.13  30.5318  1849700.0        NaN   
            2000-01-06   94.31  102.50   94.31  32.3108  2987900.0        NaN   
            2000-01-07  101.10  103.80   99.94  32.9519  2050600.0        NaN   
            2000-01-10  100.40  103.50  100.00  32.7916  1931900.0        NaN   
            2000-01-11  100.80  102.50  100.50  32.2146  1178800.0        NaN   
            2000-01-12  101.90  103.60  100.80  32.3108  1434200.0        NaN   
            2000-01-13  101.30  101.90  100.40  32.3108  1122200.0        NaN   
            2000-01-14  100.80  100.90   98.94  31.8332  1270900.0        NaN   
            2000-01-18   97.25   99.25   97.13  31.1344  1057400.0        NaN   
            2000-01-19   97.75   99.94   97.75  31.6729  1314100.0        NaN   
            2000-01-20   99.06   99.38   94.50  30.4933  1552300.0        NaN   
            2000-01-21   95.25   96.56   94.56  30.4324  2245900.0        NaN   
            2000-01-24   95.81   95.81   91.63  29.7304  1485500.0        NaN   
            2000-01-25   94.88   94.88   90.25  29.2913  1845600.0        NaN   
            2000-01-26   95.88   97.50   95.31  30.8716  3510400.0        NaN   
            2000-01-27   97.19   98.31   96.06  30.8523  1457400.0        NaN   
            2000-01-28   95.75   95.81   92.25  29.5702  1873700.0        NaN   
            2000-01-31   92.25   97.38   92.06  30.0125  1641400.0  31.031030   
            2000-02-01   94.38   96.19   93.75  30.1728  1321100.0  30.949294   
            2000-02-02   93.19   94.00   92.00  29.6503  1462100.0  30.825580   
            2000-02-03   91.94   92.13   90.00  29.3907  1760300.0  30.688925   
            2000-02-04   91.94   92.44   89.06  28.5476  1646700.0  30.484989   
            2000-02-07   89.25   89.38   86.63  28.0892  1967700.0  30.256819   
            2000-02-08   87.63   90.19   87.44  28.1085  1442000.0  30.052217   
            2000-02-09   88.63   89.00   87.50  28.0892  1300200.0  29.865263   
            2000-02-10   87.25   87.25   85.31  27.5668  2010000.0  29.646362   
            2000-02-11   85.38   86.50   81.31  26.5859  2245800.0  29.354889   
            2000-02-14   83.25   84.75   82.38  26.5667  1678200.0  29.089347   
...                        ...     ...     ...      ...        ...        ...   
            2000-09-12   88.44   89.69   88.00  28.7645  1268700.0  30.071319   
            2000-09-13   88.63   89.94   86.19  28.2549  1322000.0  29.898327   
            2000-09-14   86.25   86.44   84.13  27.7650  1973800.0  29.695153   
            2000-09-15   86.25   86.25   84.13  27.8074  2401000.0  29.515367   
            2000-09-18   84.38   84.63   82.00  27.1737  2110100.0  29.292351   
            2000-09-19   84.19   84.69   83.06  27.3991  1652700.0  29.112041   
            2000-09-20   83.75   84.69   80.50  26.7034  2260000.0  28.882647   
            2000-09-21   84.63   87.81   84.50  28.2353  2910100.0  28.820995   
            2000-09-22   87.50   89.50   86.25  28.8266  2546200.0  28.821529   
            2000-09-25   88.00   90.06   87.75  29.3982  2146900.0  28.876450   
            2000-09-26   90.00   91.00   88.44  29.3590  2932900.0  28.922407   
            2000-09-27   89.94   91.25   89.19  29.3786  2535800.0  28.965854   
            2000-09-28   90.19   93.44   90.06  29.9306  1653000.0  29.057734   
            2000-09-29   91.38   93.38   91.00  29.7673  1629100.0  29.125312   
            2000-10-02   91.94   92.69   90.81  29.9502  1246700.0  29.203873   
            2000-10-03   91.50   94.94   91.50  30.7048  1455800.0  29.346818   
            2000-10-04   93.31   96.00   93.31  31.1947  

In [11]:
#US_yields=pd.read_pickle(r'C:\Users\Student\Documents\Trading\US_yieldcurve.pkl')

In [12]:
import quandl
US_yields=quandl.get("USTREASURY/YIELD", authtoken="wdLjKc5NXiuwUhFLByKm")
US_yields=USyields[['3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '30 YR']]

NameError: name 'USyields' is not defined

In [ ]:
US_yields.index.rename('date', inplace=True)

In [ ]:
prices.reset_index(level='date',inplace=True)

In [ ]:
test=pd.merge(prices, US_yields, how='left', left_on='date' , right_index=True)

In [ ]:
test.set_index('date',append=True,inplace=True)

In [ ]:
test.sort_index(level=1, ascending=True,inplace=True)

In [ ]:
test.head()

In [25]:
### Calculated columns
#prices['EMA_20'] = prices.groupby(level=1).apply(lambda x: ta.EMA(x.close.values, 20))  # 20 day exponential moving average
prices  = test.copy().loc[(test.loc[:,'open':'30 YR'] != 0 ).all(axis=1)] # Don't touch this (:
#prices  = prices.copy().loc[(prices.loc[:,volume'] != 0 ).all(axis=1)] # Don't touch this (:

##price_close = prices.groupby(level=['symbol']).close

##prices['EMA_20'] = price_close.apply(lambda x: ta.EMA(x,20))   # 20 day exponential moving average
##prices['EMA_50'] = price_close.apply(lambda x: ta.EMA(x, 50))  # 50 day exponential moving average
##prices['EMA_200'] = price_close.apply(lambda x: ta.EMA(x, 200))  # 200 day exponential moving average
##prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
##prices['RSI'] = price_close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

#prices['CCI'] = prices.apply(lambda x: ta.CCI(high=x.high.va, low=x.low.values, close=x.close.values, timeperiod=20))  # 20 day commodity channel index

##prices['Momentum'] = price_close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum

#prices['Stoch_fastk'], prices['Stoch_fastd'] = prices.groupby(level=1).apply(lambda x: ta.STOCHF(high=x.high.values, low=x.low.values, close=x.close.values, fastk_period=14, fastd_period=3))  # Stochastic fast indicator
#prices['OBV'] = price_close.apply(lambda x: ta.OBV(np.asarray(x['close']), x['volume']))  # On Balance Volume
#prices['bbandupper'], prices['bbandmiddle'], prices['bbandlower'] =  prices.groupby(level=1).apply(lambda x: ta.BBANDS(x.close.values))


#prices['perc_change']=(prices['close']- prices['close'].shift(1))/prices['close'].shift(1)  # Getting the daily percentage change
#prices['5dayvol'] = prices['perc_change'].rolling(window=5, center=False).std()  # 5 day volatility 
#prices['5_day_target'] = prices.groupby(level=['symbol']).close.shift(-5) / prices.groupby(level=['symbol']).close  # target of X days
prices['5_day_target'] = prices.groupby(['symbol'])['close'].transform(lambda x:x.shift(-5))
prices['perc_change']=prices['5_day_target']/prices['close']
#prices['5dayvol'] = prices.groupby('symbol')['close']/prices.groupby(['symbol'])['close'].transform(lambda x:x.shift(1)).apply(pd.rolling_mean, 5, min_periods=5)  # 5 day volatility 

clean_prices = prices.loc[(prices['EMA_200'].notnull() & prices['5_day_target'].notnull())]


In [87]:
prices.head(100)

open    high     low    close     volume     EMA_20  \
symbol name date                                                                
MMM    3M   2000-01-03   96.06   96.44   94.06  30.2529  1086700.0        NaN   
            2000-01-04   92.88   94.81   90.63  29.0509  1356900.0        NaN   
            2000-01-05   91.13   95.31   91.13  30.5318  1849700.0        NaN   
            2000-01-06   94.31  102.50   94.31  32.3108  2987900.0        NaN   
            2000-01-07  101.10  103.80   99.94  32.9519  2050600.0        NaN   
            2000-01-10  100.40  103.50  100.00  32.7916  1931900.0        NaN   
            2000-01-11  100.80  102.50  100.50  32.2146  1178800.0        NaN   
            2000-01-12  101.90  103.60  100.80  32.3108  1434200.0        NaN   
            2000-01-13  101.30  101.90  100.40  32.3108  1122200.0        NaN   
            2000-01-14  100.80  100.90   98.94  31.8332  1270900.0        NaN   
            2000-01-18   97.25   99.25   97.13  31.1344  1057400.0        NaN   
            2000-01-19   97.75   99.94   97.75  31.6729  1314100.0        NaN   
            2000-01-20   99.06   99.38   94.50  30.4933  1552300.0        NaN   
            2000-01-21   95.25   96.56   94.56  30.4324  2245900.0        NaN   
            2000-01-24   95.81   95.81   91.63  29.7304  1485500.0        NaN   
            2000-01-25   94.88   94.88   90.25  29.2913  1845600.0        NaN   
            2000-01-26   95.88   97.50   95.31  30.8716  3510400.0        NaN   
            2000-01-27   97.19   98.31   96.06  30.8523  1457400.0        NaN   
            2000-01-28   95.75   95.81   92.25  29.5702  1873700.0        NaN   
            2000-01-31   92.25   97.38   92.06  30.0125  1641400.0  31.031030   
            2000-02-01   94.38   96.19   93.75  30.1728  1321100.0  30.949294   
            2000-02-02   93.19   94.00   92.00  29.6503  1462100.0  30.825580   
            2000-02-03   91.94   92.13   90.00  29.3907  1760300.0  30.688925   
            2000-02-04   91.94   92.44   89.06  28.5476  1646700.0  30.484989   
            2000-02-07   89.25   89.38   86.63  28.0892  1967700.0  30.256819   
            2000-02-08   87.63   90.19   87.44  28.1085  1442000.0  30.052217   
            2000-02-09   88.63   89.00   87.50  28.0892  1300200.0  29.865263   
            2000-02-10   87.25   87.25   85.31  27.5668  2010000.0  29.646362   
            2000-02-11   85.38   86.50   81.31  26.5859  2245800.0  29.354889   
            2000-02-14   83.25   84.75   82.38  26.5667  1678200.0  29.089347   
...                        ...     ...     ...      ...        ...        ...   
            2000-04-12   97.31   98.31   94.63  30.5236  1227300.0  29.200795   
            2000-04-13   94.63   96.38   91.75  30.5623  2076100.0  29.330462   
            2000-04-14   94.06   94.31   88.31  29.2915  3070800.0  29.326751   
            2000-04-17   89.19   90.56   88.06  28.9915  1446800.0  29.294822   
            2000-04-18   90.13   90.13   88.00  28.9915  1209600.0  29.265935   
            2000-04-19   89.63   91.88   89.13  29.1721   832600.0  29.256998   
            2000-04-20   90.44   92.31   90.00  29.7560   991900.0  29.304522   
            2000-04-24   91.25   97.63   91.25  31.4107  1779100.0  29.505110   
            2000-04-25   96.13   96.31   88.50  30.2591  4522100.0  29.576919   
            2000-04-26   93.50   93.50   87.38  28.6883  3478300.0  29.492289   
            2000-04-27   88.50   89.88   85.31  28.2045  2417600.0  29.369642   
            2000-04-28   87.44   87.56   86.00  27.9432  1473500.0  29.233790   
            2000-05-01   86.69   88.56   85.06  28.0432  1738200.0  29.120401   
            2000-05-02   86.00   87.56   85.63  28.0432  1229600.0  29.017810   
            2000-05-03   86.94   88.25   85.31  27.6787  1222600.0  28.890276   
            2000-05-04   86.19   87.69   84.75  27.5593  1383100.0  28.763516   
            2000-05-05   85.19   87.25   84.81  27.9625  

In [91]:
clean_prices['perc_change'].max()

14.798387096774196

In [28]:
test.head(500)

open    high     low    close     volume     EMA_20  \
symbol name date                                                                
MMM    3M   2000-01-03   96.06   96.44   94.06  30.2529  1086700.0        NaN   
            2000-01-04   92.88   94.81   90.63  29.0509  1356900.0        NaN   
            2000-01-05   91.13   95.31   91.13  30.5318  1849700.0        NaN   
            2000-01-06   94.31  102.50   94.31  32.3108  2987900.0        NaN   
            2000-01-07  101.10  103.80   99.94  32.9519  2050600.0        NaN   
            2000-01-10  100.40  103.50  100.00  32.7916  1931900.0        NaN   
            2000-01-11  100.80  102.50  100.50  32.2146  1178800.0        NaN   
            2000-01-12  101.90  103.60  100.80  32.3108  1434200.0        NaN   
            2000-01-13  101.30  101.90  100.40  32.3108  1122200.0        NaN   
            2000-01-14  100.80  100.90   98.94  31.8332  1270900.0        NaN   
            2000-01-18   97.25   99.25   97.13  31.1344  1057400.0        NaN   
            2000-01-19   97.75   99.94   97.75  31.6729  1314100.0        NaN   
            2000-01-20   99.06   99.38   94.50  30.4933  1552300.0        NaN   
            2000-01-21   95.25   96.56   94.56  30.4324  2245900.0        NaN   
            2000-01-24   95.81   95.81   91.63  29.7304  1485500.0        NaN   
            2000-01-25   94.88   94.88   90.25  29.2913  1845600.0        NaN   
            2000-01-26   95.88   97.50   95.31  30.8716  3510400.0        NaN   
            2000-01-27   97.19   98.31   96.06  30.8523  1457400.0        NaN   
            2000-01-28   95.75   95.81   92.25  29.5702  1873700.0        NaN   
            2000-01-31   92.25   97.38   92.06  30.0125  1641400.0  31.031030   
            2000-02-01   94.38   96.19   93.75  30.1728  1321100.0  30.949294   
            2000-02-02   93.19   94.00   92.00  29.6503  1462100.0  30.825580   
            2000-02-03   91.94   92.13   90.00  29.3907  1760300.0  30.688925   
            2000-02-04   91.94   92.44   89.06  28.5476  1646700.0  30.484989   
            2000-02-07   89.25   89.38   86.63  28.0892  1967700.0  30.256819   
            2000-02-08   87.63   90.19   87.44  28.1085  1442000.0  30.052217   
            2000-02-09   88.63   89.00   87.50  28.0892  1300200.0  29.865263   
            2000-02-10   87.25   87.25   85.31  27.5668  2010000.0  29.646362   
            2000-02-11   85.38   86.50   81.31  26.5859  2245800.0  29.354889   
            2000-02-14   83.25   84.75   82.38  26.5667  1678200.0  29.089347   
...                        ...     ...     ...      ...        ...        ...   
            2001-11-16  115.80  116.50  114.00  38.3860  2129300.0  36.670756   
            2001-11-19  115.80  117.70  115.30  38.9535  1604100.0  36.888160   
            2001-11-20  115.40  117.00  115.40  38.9869  1541800.0  37.088040   
            2001-11-21  116.30  116.30  114.90  38.7185  1263200.0  37.243322   
            2001-11-23  117.50  117.50  115.40  39.3559   589400.0  37.444520   
            2001-11-26  117.20  117.30  116.00  39.2553  1210000.0  37.616975   
            2001-11-27  116.80  116.90  115.30  38.9869  1387100.0  37.747444   
            2001-11-28  115.80  115.80  114.30  38.4500  1447900.0  37.814354   
            2001-11-29  114.60  115.80  114.20  38.6514  1353800.0  37.894073   
            2001-11-30  115.20  115.60  114.60  38.4500  1227900.0  37.947018   
            2001-12-03  114.40  114.40  112.70  38.2152  1157600.0  37.972559   
            2001-12-04  114.50  116.60  114.00  39.0540  1270600.0  38.075554   
            2001-12-05  117.00  119.50  117.00  39.8928  1579000.0  38.248625   
            2001-12-06  119.70  119.70  118.00  39.8592  1090600.0  38.402013   
            2001-12-07  118.80  118.80  117.40  39.6915  1204600.0  38.524821   
            2001-12-10  117.80  118.00  115.10  38.7856  1538000.0  38.549657   
            2001-12-11  116.30  116.70  115.30  38.8527  

In [29]:
list(clean_prices.columns.values)

['open',
 'high',
 'low',
 'close',
 'volume',
 'EMA_20',
 'EMA_50',
 'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 'bbandupper',
 'bbandmiddle',
 'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 '30 YR',
 '5_day_target',
 'perc_change']

In [30]:
testML=clean_prices.loc[:,[
 #'open',
 #'high',
 #'low',
 #'close',
 #'volume',
 #'EMA_20',
 #'EMA_50',
 #'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 #'Stoch_fastk',
 #'Stoch_fastd',
 #'OBV',
 #'bbandupper',
 #'bbandmiddle',
 #'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 #'30 YR',
'perc_change']]

In [31]:
testML[testML.isnull().any(axis=1)].head()

EMA_ratio        RSI         CCI  Momentum  \
symbol name date                                                     
MMM    3M   2001-10-08   1.033938  47.546510   17.322749    2.2631   
            2001-11-12   1.006717  61.770508   13.430441    1.5021   
            2002-10-14   0.997865  56.333230   99.186099    3.7462   
            2002-11-11   0.972856  52.864354 -146.486021    0.3406   
            2003-10-13   0.937310  65.952583   89.042918    2.9531   

                        bbandlowerii  bbandupperii  3 MO  6 MO  1 YR  2 YR  \
symbol name date                                                             
MMM    3M   2001-10-08      0.984963      1.024707   NaN   NaN   NaN   NaN   
            2001-11-12      0.984039      1.011636   NaN   NaN   NaN   NaN   
            2002-10-14      0.907886      1.023683   NaN   NaN   NaN   NaN   
            2002-11-11      0.996084      1.037461   NaN   NaN   NaN   NaN   
            2003-10-13      0.972002      1.007064   NaN   NaN   NaN   NaN   

                        3 YR  5 YR  7 YR  10 YR  perc_change  
symbol name date                                              
MMM    3M   2001-10-08   NaN   NaN   NaN    NaN     1.074658  
            2001-11-12   NaN   NaN   NaN    NaN     1.051351  
            2002-10-14   NaN   NaN   NaN    NaN     1.066115  
            2002-11-11   NaN   NaN   NaN    NaN     1.008702  
            2003-10-13   NaN   NaN   NaN    NaN     1.012881

In [32]:
testML=testML.fillna(method='pad')

In [86]:
testML.sort_values(['perc_change'], ascending=False)

EMA_ratio        RSI  \
symbol name                              date                               
MKL    Markel                            2010-04-22   1.038096   5.549272   
                                         2010-04-26   1.096488   5.636301   
                                         2010-04-21   1.009888   5.549272   
                                         2010-04-23   1.066940   5.642502   
                                         2010-04-27   1.126709   5.662613   
FMCC   Freddie Mac                       2008-09-17   3.270420  26.226594   
JCP    Farmers Insurance Exchange        2018-05-21   1.220210  34.202318   
                                         2018-05-18   1.215273  37.080806   
FMCC   Freddie Mac                       2008-09-18   3.363585  26.870298   
JCP    Farmers Insurance Exchange        2018-05-17   1.211487  39.315322   
FMCC   Freddie Mac                       2008-09-16   3.178530  26.121928   
JCP    Farmers Insurance Exchange        2018-05-15   1.210087  45.889200   
                                         2018-05-16   1.208493  53.547989   
WMB    Williams                          2002-07-25   2.156385  17.180538   
PGR    Progressive                       2006-05-12   0.995479  54.715993   
FNMA   Fannie Mae                        2008-09-17   2.819290  27.650446   
PGR    Progressive                       2006-05-15   0.994963  59.867983   
                                         2006-05-17   0.994180  57.736705   
                                         2006-05-11   0.995697  56.803834   
                                         2006-05-16   0.994554  58.059935   
FNMA   Fannie Mae                        2008-09-18   2.898887  28.082226   
AIG    AIG                               2009-03-12  11.033194  29.240253   
FNMA   Fannie Mae                        2013-03-13   0.969121  51.334181   
FMCC   Freddie Mac                       2013-03-13   0.964781  48.424773   
                                         2008-09-19   3.454544  29.306004   
FNMA   Fannie Mae                        2008-11-20   7.557998  27.297241   
AIG    AIG                               2009-03-11  10.864883  29.240253   
WMB    Williams                          2002-07-26   2.211747  19.243724   
DAN    Dana                              2009-05-01   4.360753  53.831401   
PGR    Progressive                       2002-04-26   0.842781  48.494052   
...                                                        ...        ...   
DRI    Darden Restaurants                2018-05-16   0.969501  37.070347   
WDC    Guardian Life Ins. Co. of America 2018-05-16   0.992454  57.783629   
                                         2018-05-18   0.992740  51.885247   
                                         2018-05-17   0.992293  56.450106   
HSIC   Henry Schein                      2018-05-21   1.038535  45.983288   
                                         2018-05-17   1.039423  49.256011   
                                         2018-05-18   1.038870  47.903280   
                                         2018-05-15   1.040937  49.896453   
                                         2018-05-16   1.040142  49.716120   
WDC    Guardian Life Ins. Co. of America 2018-05-15   0.992803  48.689310   
ORLY   O’Reilly Automotive               2018-05-15   0.967178  71.277017   
                                         2018-05-17   0.962890  70.546117   
                                         2018-05-16   0.965059  69.405013   
                                         2018-05-18   0.960946  69.182297   
                                         2018-05-21   0.958677  72.146954   
GILD   CHS                               2018-05-21   1.036206  40.429507   
                                         2018-05-15   1.029929  36.391625   
                                         2018-05-18   1.034727  42.009265   
                                         2018-05-16   1.031689  39.378674   
                                         2018-05-

In [33]:
    #x_train, x_test, y_train, y_test = train_test_split(clean_prices.loc[:,'EMA_ratio':'bbandlower'], clean_prices.loc[:,['7_day_target']], shuffle=False, test_size=90)
    test_size = 3
    # Get columns
    X = testML.copy().loc[:,'EMA_ratio':'10 YR'] 
    y = testML.copy().loc[:, ['perc_change']]

In [34]:
    # Get series of ingroup rownumbers
    X['num']= X.groupby(level=1).cumcount(ascending=False)
    
    testML.loc[ testML.EMA_ratio == testML.EMA_ratio.max() ] 

EMA_ratio        RSI  \
symbol name                             date                               
NFLX   Netflix                          2003-01-22  40.748452  53.007067   
       Thrivent Financial for Lutherans 2003-01-22  40.748452  53.007067   

                                                          CCI  Momentum  \
symbol name                             date                              
NFLX   Netflix                          2003-01-22 -41.691894     0.045   
       Thrivent Financial for Lutherans 2003-01-22 -41.691894     0.045   

                                                    bbandlowerii  \
symbol name                             date                       
NFLX   Netflix                          2003-01-22      0.956294   
       Thrivent Financial for Lutherans 2003-01-22      0.956294   

                                                    bbandupperii  3 MO  6 MO  \
symbol name                             date                                   
NFLX   Netflix                          2003-01-22      1.065671  1.17  1.19   
       Thrivent Financial for Lutherans 2003-01-22      1.065671  1.17  1.19   

                                                    1 YR  2 YR  3 YR  5 YR  \
symbol name                             date                                 
NFLX   Netflix                          2003-01-22  1.31  1.66  2.09  2.94   
       Thrivent Financial for Lutherans 2003-01-22  1.31  1.66  2.09  2.94   

                                                    7 YR  10 YR  perc_change  
symbol name                             date                                  
NFLX   Netflix                          2003-01-22  3.49   3.95     1.081825  
       Thrivent Financial for Lutherans 2003-01-22  3.49   3.95     1.081825

In [35]:
    # Filter
    X_train = X.loc[X.num > test_size ].drop(columns='num')
    X_test =  X.loc[X.num <= test_size ].drop(columns='num')
    y_train = y.loc[X.num > test_size ]
    y_test =  y.loc[X.num <= test_size ]
    
    X_test.head(25)

EMA_ratio        RSI         CCI  \
symbol name                   date                                           
MMM    3M                     2018-05-09   1.017004  43.134255   99.576427   
                              2018-05-10   1.018187  45.983100  103.849913   
                              2018-05-11   1.019292  46.217344   81.839948   
                              2018-05-14   1.020254  47.352412   87.777878   
AMRK   A-Mark Precious Metals 2018-05-09   1.078558  47.487678 -159.889558   
                              2018-05-10   1.076514  53.555796  -45.454545   
                              2018-05-11   1.074620  53.006100   29.482551   
                              2018-05-14   1.073365  48.865097  -20.361991   
ABM    ABM Industries         2018-05-09   1.122922  31.358218  143.589744   
                              2018-05-10   1.124333  31.006196   54.112554   
                              2018-05-11   1.125617  31.420736   25.641026   
                              2018-05-14   1.126926  30.246468  -20.202020   
ADP    ADP                    2018-05-09   0.955315  73.435166  108.103896   
                              2018-05-10   0.953581  74.014323   95.485274   
                              2018-05-11   0.951972  73.766902   64.925044   
                              2018-05-14   0.950622  70.420488   11.395705   
ACM    AECOM                  2018-05-09   1.000855  41.315491   33.333566   
                              2018-05-10   1.002245  38.525495  -71.745336   
                              2018-05-11   1.004111  34.360765 -114.203629   
                              2018-05-14   1.006136  32.664164  -96.556218   
AES    AES                    2018-05-09   0.956482  58.522568  -60.470325   
                              2018-05-10   0.954856  63.322049   34.679587   
                              2018-05-11   0.953385  62.671239   63.044406   
                              2018-05-14   0.951871  63.870910   74.981142   
AGCO   AGCO                   2018-05-09   1.032947  49.753829   92.159166   

                                          Momentum  bbandlowerii  \
symbol name                   date                                 
MMM    3M                     2018-05-09    4.5491      0.958069   
                              2018-05-10    7.8865      0.964924   
                              2018-05-11    8.9790      0.971578   
                              2018-05-14   11.2934      0.978305   
AMRK   A-Mark Precious Metals 2018-05-09   -0.9400      0.993876   
                              2018-05-10   -0.5300      0.970156   
                              2018-05-11   -0.6400      0.974566   
                              2018-05-14   -0.3100      0.987356   
ABM    ABM Industries         2018-05-09   -1.6400      0.988978   
                              2018-05-10   -1.4500      0.991303   
                              2018-05-11   -0.9000      0.991069   
                              2018-05-14   -0.8400      0.999884   
ADP    ADP                    2018-05-09   10.3400      0.966749   
                              2018-05-10    8.7000      0.977905   
                              2018-05-11    7.4600      0.981969   
                              2018-05-14    8.6000      0.991920   
ACM    AECOM                  2018-05-09   -1.2000      0.976625   
                              2018-05-10   -1.0400      0.985286   
                              2018-05-11   -1.7900      0.988077   
                              2018-05-14   -1.4800      0.978210   
AES    AES                    2018-05-09    0.1664      0.967855   
                              2018-05-10    0.2679      0.948952   
                              2018-05-11    0.1094      0.950678   
                              2018-05-14    0.1100      0.946313   
AGCO   AGCO                   2018-05-09   -0.1197      0.920665   

                                          bbandupperii  3 MO  6 MO  1 YR  \
symbol name           

In [83]:
y_train.min()

ValueError: No axis named 6 for object type <class 'pandas.core.frame.DataFrame'>

In [78]:
y_train.head()

perc_change
symbol name date                   
MMM    3M   2000-10-16     0.974184
            2000-10-17     0.988259
            2000-10-18     0.989028
            2000-10-19     1.035843
            2000-10-20     1.071635

In [93]:
y_train.sort_values(['perc_change'], axis=1)

KeyError: 'perc_change'

In [36]:
X_train.head()

EMA_ratio        RSI         CCI  Momentum  \
symbol name date                                                     
MMM    3M   2000-10-16   0.977816  53.485608   26.875864    0.1633   
            2000-10-17   0.977865  50.065508  -25.001901   -0.9375   
            2000-10-18   0.977920  50.065508  -87.950702   -1.4274   
            2000-10-19   0.978649  43.571262 -109.570935   -1.3883   
            2000-10-20   0.979902  39.085852 -140.782875   -1.6136   

                        bbandlowerii  bbandupperii  3 MO  6 MO  1 YR  2 YR  \
symbol name date                                                             
MMM    3M   2000-10-16      0.977027      1.020805  6.28  6.28  5.97  5.89   
            2000-10-17      0.986199      1.026090  6.30  6.27  5.92  5.83   
            2000-10-18      0.991149      1.011834  6.30  6.26  5.91  5.82   
            2000-10-19      0.997906      1.044567  6.32  6.30  5.96  5.86   
            2000-10-20      0.992356      1.073890  6.31  6.29  5.94  5.84   

                        3 YR  5 YR  7 YR  10 YR  
symbol name date                                 
MMM    3M   2000-10-16  5.83  5.77  5.84   5.74  
            2000-10-17  5.76  5.68  5.77   5.68  
            2000-10-18  5.74  5.68  5.76   5.66  
            2000-10-19  5.77  5.69  5.76   5.66  
            2000-10-20  5.76  5.69  5.73   5.64

In [37]:
    %timeit
    forest = RandomForestRegressor()
    forest.fit(X_train,y_train)
    y_pred = forest.predict(X_test)
    y_pred = pd.DataFrame({'ans': y_pred})
    

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
test.head()

open    high    low    close     volume  EMA_20  \
symbol name date                                                            
MMM    3M   2000-01-03   96.06   96.44  94.06  30.2529  1086700.0     NaN   
            2000-01-04   92.88   94.81  90.63  29.0509  1356900.0     NaN   
            2000-01-05   91.13   95.31  91.13  30.5318  1849700.0     NaN   
            2000-01-06   94.31  102.50  94.31  32.3108  2987900.0     NaN   
            2000-01-07  101.10  103.80  99.94  32.9519  2050600.0     NaN   

                        EMA_50  EMA_200  EMA_ratio  RSI  ...    bbandupperii  \
symbol name date                                         ...                   
MMM    3M   2000-01-03     NaN      NaN        NaN  NaN  ...             NaN   
            2000-01-04     NaN      NaN        NaN  NaN  ...             NaN   
            2000-01-05     NaN      NaN        NaN  NaN  ...             NaN   
            2000-01-06     NaN      NaN        NaN  NaN  ...             NaN   
            2000-01-07     NaN      NaN        NaN  NaN  ...        1.027636   

                        3 MO  6 MO  1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  30 YR  
symbol name date                                                                
MMM    3M   2000-01-03  5.48  5.81  6.09  6.38  6.42  6.50  6.65   6.58   6.61  
            2000-01-04  5.43  5.75  6.00  6.30  6.34  6.40  6.56   6.49   6.53  
            2000-01-05  5.44  5.74  6.05  6.38  6.43  6.51  6.68   6.62   6.64  
            2000-01-06  5.41  5.69  6.03  6.35  6.39  6.46  6.63   6.57   6.58  
            2000-01-07  5.38  5.66  6.00  6.31  6.35  6.42  6.58   6.52   6.55  

[5 rows x 26 columns]

In [224]:
#final = pd.DataFrame({'close':test.tail(90).reset_index().close, 'ratio':y_pred['ans']})
#final['pred_price']= final['close'] * final['ratio']

In [45]:
testML.head()

EMA_ratio        RSI         CCI  Momentum  \
symbol name date                                                     
MMM    3M   2000-10-16   0.977816  53.485608   26.875864    0.1633   
            2000-10-17   0.977865  50.065508  -25.001901   -0.9375   
            2000-10-18   0.977920  50.065508  -87.950702   -1.4274   
            2000-10-19   0.978649  43.571262 -109.570935   -1.3883   
            2000-10-20   0.979902  39.085852 -140.782875   -1.6136   

                        bbandlowerii  bbandupperii  3 MO  6 MO  1 YR  2 YR  \
symbol name date                                                             
MMM    3M   2000-10-16      0.977027      1.020805  6.28  6.28  5.97  5.89   
            2000-10-17      0.986199      1.026090  6.30  6.27  5.92  5.83   
            2000-10-18      0.991149      1.011834  6.30  6.26  5.91  5.82   
            2000-10-19      0.997906      1.044567  6.32  6.30  5.96  5.86   
            2000-10-20      0.992356      1.073890  6.31  6.29  5.94  5.84   

                        3 YR  5 YR  7 YR  10 YR  perc_change  
symbol name date                                              
MMM    3M   2000-10-16  5.83  5.77  5.84   5.74     0.974184  
            2000-10-17  5.76  5.68  5.77   5.68     0.988259  
            2000-10-18  5.74  5.68  5.76   5.66     0.989028  
            2000-10-19  5.77  5.69  5.76   5.66     1.035843  
            2000-10-20  5.76  5.69  5.73   5.64     1.071635

In [43]:
ml=test.sort_index(level=1, ascending=True)
ml=test.groupby(['name']).tail(1).loc[:,'EMA_ratio':'10 YR']  #tail to get the last date entry for each day

In [44]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
symbol,name,date,,,,,,,,,,,,,,,,,
MMM,3M,2018-05-21,1.027037,45.770639,49.303689,2.8502,202.478565,200.1971,197.915635,0.980946,1.003561,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
AMRK,A-Mark Precious Metals,2018-05-21,1.066445,50.944985,93.745309,-0.1600,13.579333,13.2200,12.860667,0.970616,1.024855,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
ABM,ABM Industries,2018-05-21,1.133097,32.279824,-88.414634,-0.1200,30.465837,30.0740,29.682163,0.988746,1.014851,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
ADP,ADP,2018-05-21,0.943618,74.537993,110.319246,3.2700,129.511012,128.1660,126.820988,0.984329,1.005208,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
ACM,AECOM,2018-05-21,1.013332,39.804889,-28.078310,-1.1800,33.807218,33.3880,32.968782,0.988569,1.013710,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06


In [46]:
ml[ml.isnull().any(axis=1)].head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
symbol,name,date,,,,,,,,,,,,,,,,,
BRKA,Berkshire Hathaway,2018-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06


In [47]:
ml=ml.loc[ml['EMA_ratio'].notnull()]

In [48]:
ml.drop(['bbandupper','bbandmiddle','bbandlower'], axis=1, inplace=True)

In [49]:
ml['5dayforecast'] = forest.predict(ml)

In [50]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,5dayforecast
symbol,name,date,,,,,,,,,,,,,,,
MMM,3M,2018-05-21,1.027037,45.770639,49.303689,2.8502,0.980946,1.003561,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.478512
AMRK,A-Mark Precious Metals,2018-05-21,1.066445,50.944985,93.745309,-0.1600,0.970616,1.024855,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.434409
ABM,ABM Industries,2018-05-21,1.133097,32.279824,-88.414634,-0.1200,0.988746,1.014851,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.529692
ADP,ADP,2018-05-21,0.943618,74.537993,110.319246,3.2700,0.984329,1.005208,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.559747
ACM,AECOM,2018-05-21,1.013332,39.804889,-28.078310,-1.1800,0.988569,1.013710,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.483178


In [51]:
prices.head()

open    high    low    close     volume  EMA_20  \
symbol name date                                                            
MMM    3M   2000-01-03   96.06   96.44  94.06  30.2529  1086700.0     NaN   
            2000-01-04   92.88   94.81  90.63  29.0509  1356900.0     NaN   
            2000-01-05   91.13   95.31  91.13  30.5318  1849700.0     NaN   
            2000-01-06   94.31  102.50  94.31  32.3108  2987900.0     NaN   
            2000-01-07  101.10  103.80  99.94  32.9519  2050600.0     NaN   

                        EMA_50  EMA_200  EMA_ratio  RSI     ...       6 MO  \
symbol name date                                            ...              
MMM    3M   2000-01-03     NaN      NaN        NaN  NaN     ...       5.81   
            2000-01-04     NaN      NaN        NaN  NaN     ...       5.75   
            2000-01-05     NaN      NaN        NaN  NaN     ...       5.74   
            2000-01-06     NaN      NaN        NaN  NaN     ...       5.69   
            2000-01-07     NaN      NaN        NaN  NaN     ...       5.66   

                        1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  30 YR  \
symbol name date                                                     
MMM    3M   2000-01-03  6.09  6.38  6.42  6.50  6.65   6.58   6.61   
            2000-01-04  6.00  6.30  6.34  6.40  6.56   6.49   6.53   
            2000-01-05  6.05  6.38  6.43  6.51  6.68   6.62   6.64   
            2000-01-06  6.03  6.35  6.39  6.46  6.63   6.57   6.58   
            2000-01-07  6.00  6.31  6.35  6.42  6.58   6.52   6.55   

                        5_day_target  perc_change  
symbol name date                                   
MMM    3M   2000-01-03       32.7916     1.083916  
            2000-01-04       32.2146     1.108902  
            2000-01-05       32.3108     1.058267  
            2000-01-06       32.3108     1.000000  
            2000-01-07       31.8332     0.966051  

[5 rows x 28 columns]

In [52]:
result=ml.join(prices, lsuffix='l',how ='left')

In [53]:
result[['close','5dayforecast']]

,,,close,5dayforecast
symbol,name,date,,
MMM,3M,2018-05-21,201.76,0.478512
AMRK,A-Mark Precious Metals,2018-05-21,13.25,0.434409
ABM,ABM Industries,2018-05-21,30.02,0.529692
ADP,ADP,2018-05-21,128.84,0.559747
ACM,AECOM,2018-05-21,33.35,0.483178
AES,AES,2018-05-21,12.06,0.528485
AGCO,AGCO,2018-05-21,68.05,0.280792
AIG,AIG,2018-05-21,55.29,0.376636
AKS,AK Steel Holding,2018-05-21,4.46,0.516800


In [54]:
result=result[result.close.notnull()]

In [55]:
result=result[['close','5dayforecast']]

In [56]:
result.head()

,,,close,5dayforecast
symbol,name,date,,
MMM,3M,2018-05-21,201.76,0.478512
AMRK,A-Mark Precious Metals,2018-05-21,13.25,0.434409
ABM,ABM Industries,2018-05-21,30.02,0.529692
ADP,ADP,2018-05-21,128.84,0.559747
ACM,AECOM,2018-05-21,33.35,0.483178


In [57]:
result['5day_price']=result['close']*result['5dayforecast']

In [58]:
result['5day%_change']=(result['5dayforecast']-1)*100

In [59]:
result

,,,close,5dayforecast,5day_price,5day%_change
symbol,name,date,,,,
MMM,3M,2018-05-21,201.76,0.478512,96.544669,-52.148756
AMRK,A-Mark Precious Metals,2018-05-21,13.25,0.434409,5.755920,-56.559097
ABM,ABM Industries,2018-05-21,30.02,0.529692,15.901360,-47.030779
ADP,ADP,2018-05-21,128.84,0.559747,72.117768,-44.025328
ACM,AECOM,2018-05-21,33.35,0.483178,16.113996,-51.682172
AES,AES,2018-05-21,12.06,0.528485,6.373527,-47.151516
AGCO,AGCO,2018-05-21,68.05,0.280792,19.107894,-71.920802
AIG,AIG,2018-05-21,55.29,0.376636,20.824190,-62.336426
AKS,AK Steel Holding,2018-05-21,4.46,0.516800,2.304930,-48.319963


In [60]:
prep = result[['close','5day_price','5day%_change']]
prep.sort_values("5day%_change",ascending=False, inplace=True)
prep.reset_index(inplace=True)
prep.set_index('name')
tdf=timedelta(days=5)
prep=prep[prep['date']>=(datetime.now()-tdf)]

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
prep.shape

(481, 6)

In [62]:
prep.tail(20)

,symbol,name,date,close,5day_price,5day%_change
477,HTZ,Hertz Global Holdings,2018-05-21,16.52,4.653036,-71.833920
478,AGCO,AGCO,2018-05-21,68.05,19.107894,-71.920802
479,SNX,Synnex,2018-05-21,109.90,30.762700,-72.008463
480,AMGN,Amgen,2018-05-21,176.47,48.618323,-72.449525
481,RJF,Raymond James Financial,2018-05-21,100.64,27.717510,-72.458754
482,QCOM,Qualcomm,2018-05-21,57.71,15.848222,-72.538170
483,APA,Airgas,2018-05-21,44.28,12.129832,-72.606523
484,APA,Apache,2018-05-21,44.28,12.129832,-72.606523
485,ESRX,Express Scripts Holding,2018-05-21,76.52,20.732713,-72.905498
486,WHR,Whirlpool,2018-05-21,164.95,44.682412,-72.911542


In [63]:
prep.head(20)

,symbol,name,date,close,5day_price,5day%_change
0,SPTN,SpartanNash,2018-05-21,18.50,68.902798,272.447558
1,GE,General Electric,2018-05-21,15.26,50.709293,232.302048
2,CYH,Community Health Systems,2018-05-21,4.58,15.205080,231.988638
3,WIN,Windstream Holdings,2018-05-21,1.48,4.371523,195.373147
4,FMCC,Freddie Mac,2018-05-21,1.47,4.260997,189.863723
5,JCP,Farmers Insurance Exchange,2018-05-21,2.50,7.199016,187.960646
6,JCP,J.C. Penney,2018-05-21,2.50,7.199016,187.960646
7,FTR,Frontier Communications,2018-05-21,9.60,27.628945,187.801508
8,BBBY,Bed Bath & Beyond,2018-05-21,18.01,51.460882,185.735047
9,ODP,Office Depot,2018-05-21,2.41,6.019028,149.752213
